In [2]:
# 声明库依赖
import pickle
import numpy as np
import tensorflow as tf
import os
import time

image_size = 28
num_labels = 10

class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()

        pickle_file = os.getcwd() + '/../../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# 1.两层神经网络--隐藏层拥有512个神经元

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
# 非线性激活函数LeRU
hiddle = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle, w2) + b2)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './2hiddle256-model_1.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.758826530
Epoch: 0002 cost= 0.591107621
Epoch: 0003 cost= 0.556157197
Epoch: 0004 cost= 0.533507502
Optimization Finished!
Accuracy: 0.9125


In [10]:
# 2.两层神经网络--隐藏层拥有512个神经元
not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hiddle = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))
# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle, w2) + b2)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './2hiddle512-model_1.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.722657352
Epoch: 0002 cost= 0.572988732
Epoch: 0003 cost= 0.534855298
Epoch: 0004 cost= 0.510077099
Optimization Finished!
Accuracy: 0.9191


In [11]:
# 3.三层神经网络

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle2, w3) + b3)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_1.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.685514792
Epoch: 0002 cost= 0.537310714
Epoch: 0003 cost= 0.494885974
Epoch: 0004 cost= 0.466633844
Optimization Finished!
Accuracy: 0.9239


In [12]:
# 4.三层神经网络--Tenorflow内置交叉商检验
not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
# 第一层隐藏单元
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# 第二层隐藏单元
w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

# 输出层
w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_2.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 0001 cost= 0.691741411
Epoch: 0002 cost= 0.540446746
Epoch: 0003 cost= 0.498011492
Epoch: 0004 cost= 0.469659418
Optimization Finished!
Accuracy: 0.929


In [5]:
# 5.三层神经网络--ADAM优化器来做梯度最速下降
import time
not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量
    t0 = time.time()
    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        t1 = time.time()
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            print("Epoch:", '%04d' % (epoch + 1), ' epoch time:', time.time() - t1)
    print('fit time:', time.time() - t0)
    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_3.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.599719571
Epoch: 0001  epoch time: 30.024717330932617
Epoch: 0002 cost= 0.574741803
Epoch: 0002  epoch time: 25.346449613571167
Epoch: 0003 cost= 0.556158303
Epoch: 0003  epoch time: 24.724414110183716
Epoch: 0004 cost= 0.545725026
Epoch: 0004  epoch time: 24.29038953781128
fit time: 104.38797044754028
Optimization Finished!
Accuracy: 0.9018


In [ ]:
# 6.三层神经网络--使用dropout

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)
# 一般只有训练的时候才使用
hiddle1 = tf.nn.dropout(hiddle1, 0.9)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle2, w3) + b3)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量
    t0 = time.time()
    # 迭代训练
    for epoch in range(training_epochs):
        t1 = time.time()
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            print("Epoch:", '%04d' % (epoch + 1), ' epoch time:', time.time() - t1)
    print('fit time:', time.time() - t0)
    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_4.ckpt')

In [3]:
# 7.三层神经网络--使用dropout和regularization
import time

print('DNN begin')

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))

hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)
hiddle1 = tf.nn.dropout(hiddle1, 0.9)


w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))

# 一般只有训练的时候才使用
cost = cost + 5e-4 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) 
                      + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2) 
                      + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量
    t0 = time.time()
    # 迭代训练
    for epoch in range(training_epochs):
        t1 = time.time()
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            print("Epoch:", '%04d' % (epoch + 1), ' epoch time:', time.time() - t1)
    print('fit time:', time.time() - t0)
    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_5.ckpt')

DNN begin
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 0001 cost= 1.758224982
Epoch: 0001  epoch time: 30.200666904449463
Epoch: 0002 cost= 1.577368458
Epoch: 0002  epoch time: 34.661213874816895
Epoch: 0003 cost= 1.516258265
Epoch: 0003  epoch time: 28.758651733398438
Epoch: 0004 cost= 1.470628965
Epoch: 0004  epoch time: 23.91585087776184
fit time: 117.53738355636597
Optimization Finished!
Accuracy: 0.9175


In [4]:
# 8.三层神经网络--使用dropout和regularization
import time
not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)
# 一般只有训练的时候才使用
hiddle1 = tf.nn.dropout(hiddle1, 0.9)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))

# 一般只有训练的时候才使用
cost = cost + 5e-4 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) 
                      + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2) 
                      + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量
    t0 = time.time()
    # 迭代训练
    for epoch in range(training_epochs):
        t1 = time.time()
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            print("Epoch:", '%04d' % (epoch + 1), ' epoch time:', time.time() - t1)
    print('fit time:', time.time() - t0)

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, './3hiddle512-model_6.ckpt')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 1.753988007
Epoch: 0001  epoch time: 33.93756914138794
Epoch: 0002 cost= 1.573249657
Epoch: 0002  epoch time: 36.992921352386475
Epoch: 0003 cost= 1.511219210
Epoch: 0003  epoch time: 32.46866488456726
Epoch: 0004 cost= 1.464550848
Epoch: 0004  epoch time: 31.060059070587158
fit time: 134.47581481933594
Optimization Finished!
Accuracy: 0.9156
